## Testing ConllDataset

In [1]:
from src.dataset import ConllDataset

original_file = './data/wsj_dev.conll06.pred'
conll_dataset = ConllDataset(original_file)

Test writing to new file

In [2]:
conll_dataset.write(filepath='./test.conll06')

## Access sentence

In [3]:
from src.dataset import ConllDataset

original_file = './data/wsj_dev.conll06.pred'
conll_dataset = ConllDataset(original_file)

In [4]:
conll_dataset[0].sentence

,ID,FORM,LEMMA,POS,XPOS,MORPH.,HEAD,REL,DEPS,MISC
0,1,The,the,DT,_,_,2,NMOD,_,_
1,2,economy,economy,NN,_,_,4,NMOD,_,_
2,3,'s,'s,POS,_,_,2,NMOD,_,_
3,4,temperature,temperature,NN,_,_,5,SBJ,_,_
4,5,will,will,MD,_,_,0,ROOT,_,_
5,6,be,be,VB,_,_,5,VC,_,_
6,7,taken,take,VBN,_,_,6,VC,_,_
7,8,from,from,IN,_,_,7,ADV,_,_
8,9,several,several,JJ,_,_,11,NMOD,_,_
9,10,vantage,vantage,NN,_,_,11,NMOD,_,_


## Evaluation

In [ ]:
from src.dataset import ConllDataset

gold_file = './data/wsj_dev.conll06.gold'
prediction_file = './data/wsj_dev.conll06.pred'

conll_gold = ConllDataset(gold_file)
conll_prediction = ConllDataset(prediction_file)

In [ ]:
from src.evaluation import uas, las

In [ ]:
uas(conll_gold, conll_prediction)

0.8974243278811807

In [ ]:
las(conll_gold, conll_prediction)

0.8811430720060162